In [37]:
import sklearn
import numpy as np
import pandas as pd

In [38]:
from numpy.random import RandomState

df = pd.read_csv('Twitter_Data.csv')
rng = RandomState()

train = df.sample(frac=0.25, random_state=rng)
# test = df.loc[~df.index.isin(train.index)]
test = df.sample(frac=0.10, random_state=rng)

In [39]:
train.head()

,clean_text,category
39464,you will win behalf modi,1.0
122152,come guys though modi deserves more but photos...,1.0
60900,you maybe right\njust help understand iaf stri...,1.0
84976,since bjpites have been jumping saying modi ne...,0.0
153208,full interview watch prime minister narendra m...,1.0


In [40]:
test.head()

,clean_text,category
61395,you are not journalist you are biased modi sup...,0.0
141239,rajnit big aal raundar manniny bhavi narendar ...,1.0
39442,when authoritarians face elcewtion they wag th...,0.0
72311,what there talk they dont grow trees took year...,1.0
136653,yaa too\nand yes modi,0.0


In [41]:
sum(train["category"] == 1)

17911

In [42]:
sum(train["category"] == 0)

13934

In [43]:
sum(train["category"] == -1)

8898

In [44]:
train.isnull().sum()

clean_text    0
category      2
dtype: int64

In [45]:
!pip install tweet-preprocessor


In [46]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [47]:
train = train.dropna()
test = test.dropna()

In [48]:
train.isnull().sum()

clean_text    0
category      0
dtype: int64

In [49]:
test.isnull().sum()

clean_text    0
category      0
dtype: int64

In [50]:
sum(train["category"] == 1)

17911

In [51]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr


In [52]:
# train_tweet = clean_tweets(train["clean_text"])
# train_tweet = pd.DataFrame(train_tweet)

In [53]:
# append cleaned tweets to the training data
# train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,clean_text,category
39464,you will win behalf modi,1.0
122152,come guys though modi deserves more but photos...,1.0
60900,you maybe right\njust help understand iaf stri...,1.0
84976,since bjpites have been jumping saying modi ne...,0.0
153208,full interview watch prime minister narendra m...,1.0
77843,surgical strikes iaf airstrikes inside pakista...,0.0
73603,india under leadership despotic cruel dictator...,-1.0
4359,childlike innocence sidiqul farooq has made fa...,0.0
130345,she must fight against modi sure she may give ...,1.0
127768,till 2014 scientists were supposed keep quiet ...,1.0


In [54]:
# clean the test data and append the cleaned tweets to the test data
# test_tweet = clean_tweets(test["clean_text"])
# test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
# test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.head()


,clean_text,category
61395,you are not journalist you are biased modi sup...,0.0
141239,rajnit big aal raundar manniny bhavi narendar ...,1.0
39442,when authoritarians face elcewtion they wag th...,0.0
72311,what there talk they dont grow trees took year...,1.0
136653,yaa too\nand yes modi,0.0


In [55]:
train = train.dropna()
test = test.dropna()

In [56]:
test.tail()

,clean_text,category
29619,modi has been loyal his financiers the corpora...,1.0
24034,bbc news nirav modi billionaires art auctioned,0.0
72115,but the credit goes one and only refused earli...,0.0
14398,read manifesto 2014 mention lakh rupees offici...,0.0
147658,chokidaar means modi and the biggest thief mil...,0.0


In [58]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.category.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_text.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer


In [69]:
documents = ["This is Koushik Joshi and I am from Jain University"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

,am,and,from,is,jain,joshi,koushik,this,university
0,1,1,1,1,1,1,1,1,1


In [61]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)


In [62]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [63]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  82.05841446453408 %


In [64]:
print("done")

done


In [65]:
import pickle
import joblib
joblib.dump(svm, 'finalized_svm_model.sav')

['finalized_svm_model.sav']